In [ ]:
import re
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.auto import *
import scanpy as sc
import anndata
import pandas as pd
import numpy as np
import warnings
import matplotlib as mpl
import torch
import random
import pyBigWig
import numpy as np
import pandas as pd
import re
from tqdm.auto import trange, tqdm
import scprinter as scp
import argparse
import os
import pyBigWig
torch.set_num_threads(4)

mpl.rcParams['pdf.fonttype'] = 42
import bioframe

In [ ]:
models = ['HepG2_fold0-crimson-bao-8.pt',
'HepG2_fold1-auspicious-rocket-9.pt',
'HepG2_fold2-prosperous-fish-8.pt',
'HepG2_fold3-prosperous-bao-11.pt',
'HepG2_fold4-radiant-paper-12.pt']

In [5]:
def read_TF_loci(path):
    TF_prediction = pd.read_csv(path, sep='\t')
    if 'range' in TF_prediction.columns:
        ranges = pd.DataFrame([re.split(':|-', xx) for xx in TF_prediction['range']])
        ranges = ranges[[0,1,2,3]]
        v = np.array(ranges[3])
        v[v == ''] = '-'
        ranges[3] = v
        ranges.columns = ['chrom','start','end', 'strand']
        ranges['start'] = ranges['start'].astype('int')
        ranges['end'] = ranges['end'].astype('int')
        TF_prediction = pd.concat([TF_prediction, ranges], axis=1)
    else:
        TF_prediction = pd.read_csv(path, sep='\t', header=None)
        TF_prediction.columns = ['chrom','start','end', 'bound', 'TF']
    # TF_prediction = TF_prediction.sort_values(by=['bound'], ascending=False) # sort such that bound = 1, comes first, and will be kept.
    TF_prediction['summit'] = (TF_prediction['start'] + TF_prediction['end']) // 2
    return TF_prediction
TFBS_path = "/data/rzhang/PRINT_rev/HepG2/TF_prediction.tsv"
TFBS_old = read_TF_loci(TFBS_path)
TFBS_old = TFBS_old[TFBS_old['bound'] == 1]
TFBS_old = TFBS_old[['chrom','start','end','strand', 'TF']]
TFBS_old['start'] -= 1

In [6]:
TFBS = pd.read_csv("/data/rzhang/PRINT_rev/HepG2/filteredTFBS.tsv", sep='\t', header=None)
TFBS.columns = ['chrom','start','end', 'strand', 'TF']
TFBS_all = pd.concat([TFBS_old, TFBS], axis=0)
TFBS_all = TFBS_all.drop_duplicates()

In [ ]:
TFBS = TFBS_all
# TFBS.columns = ['chrom','start','end','strand', 'TF']
print (TFBS)
dfs = TFBS
## planting
for model in models:
    save_path = os.path.join("/data/rzhang/PRINT_rev/HepG2/final_model",  f"{model}_diff_results.npy")
    model = torch.load(os.path.join("/data/rzhang/PRINT_rev/HepG2/final_model", model), map_location='cpu').cuda()
    import copy
    import random
    
    diff_results = []
    with torch.no_grad():
        with (torch.autocast(device_type='cuda', dtype=torch.float16, enabled=True)):
            seqs = []
            seq2s = []
            for i in trange(len(dfs)):
                chr = dfs['chrom'][i]
                left = int(dfs['start'][i]) #- 1
                right = int(dfs['end'][i])
    
    
                center = (left + right) // 2
                flank = 920
                seq = scp.genome.hg38.fetch_seq(chr,
                                                center - flank,
                                                center + flank)
                seq2 = list(seq)
    
                size = right - left + 4
                seq2[left - 2 - (center - flank): right + 2 - (center - flank)] = random.choices(seq[left - 2 - (center - flank): right + 2 - (center - flank)],
                                                                                                 k=size)
    
                seq2 = ''.join(seq2)
                seq = scp.utils.DNA_one_hot(seq)[None].float()#.cuda()
                seq2 = scp.utils.DNA_one_hot(seq2)[None].float()#.cuda()
    
                seqs.append(seq)
                seq2s.append(seq2)
    
                if len(seqs) == 256:
                    seqs = torch.cat(seqs, dim=0).cuda()
                    seq2s = torch.cat(seq2s, dim=0).cuda()
                    pred = scp.utils.zscore2pval_torch(model(seqs)[0])
                    pred_mut = scp.utils.zscore2pval_torch(model(seq2s)[0])
    
                    diff = (pred_mut - pred)
                    diff = diff[..., ::2]
                    # print (diff.shape)
                    diff_results.append(diff.detach().cpu().numpy())
                    seqs = []
                    seq2s = []
    
            if len(seqs) > 0:
                seqs = torch.cat(seqs, dim=0).cuda()
                seq2s = torch.cat(seq2s, dim=0).cuda()
                pred = scp.utils.zscore2pval_torch(model(seqs)[0])
                pred_mut = scp.utils.zscore2pval_torch(model(seq2s)[0])
    
                diff = (pred_mut - pred)
                diff = diff[..., ::2]
                diff_results.append(diff.detach().cpu().numpy())
    
    
    diff_results = np.concatenate(diff_results, axis=0)
    print (diff_results.shape)
    np.save(save_path, diff_results)


In [ ]:
all = 0
for m in models:
    all += np.load(f"/data/rzhang/PRINT_rev/HepG2/final_model/{m}_diff_results.npy")
all /= 5
np.savez("/data/rzhang/PRINT_rev/HepG2/final_model/avg_diff_results.npy", all)